In [1]:
import requests
from requests.exceptions import HTTPError

import json
import datetime
import pandas as pd

In [2]:
# Grunnupplýsingar um deildina
league_id = '1150562'

# end points
url = 'https://fantasy.premierleague.com/api/'

leagues_endpoint = f'leagues-classic/{1150562}/standings/'
entry_endpoint = 'entry/'
static_endpoint = 'bootstrap-static/'

### Hjálparföll

In [3]:
# prentar út json object fallega
def print_json(json_object):
    print(json.dumps(json_object, indent=2, ensure_ascii=False))

### FPL föll sem kalla í apann

In [4]:
def find_current_gw():
    try:
        response = requests.get(url + static_endpoint)
        
        events = response.json()['events']
        for event in events:
            if event['is_current']:
                return int(event['id'])
        
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        return -1
    except Exception as err:
        print(f'Other error occurred: {err}')
        return -1

In [5]:
def is_gw_finished(gw):
    try:
        response = requests.get(url + static_endpoint)
        
        events = response.json()['events']
        for event in events:
            if event['id'] == gw:
                return event['finished']
        
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        return None
    except Exception as err:
        print(f'Other error occurred: {err}')
        return None

In [6]:
def find_current_phase():
    for key in phases:
        phase = phases[key]
        if phase['is_current']:
            return int(phase['id'])

In [7]:
def is_phase_finished(phase_id):
    for key in phases:
        phase = phases[key]
        end = phase['end_event']
        return is_gw_finished(end)

In [8]:
# finnur id fyrir hvern og einn leikmann í deildinni, skilar dict
def find_gulgast_entries_id():
    try:
        response = requests.get(url + leagues_endpoint)
        
        # kóðinn fer hingað ef það er engin villa
        league_json = response.json()
        entries_json = league_json['standings']['results']
        gulgast_entries = {}
        for entry in entries_json:
            entry_id = entry['entry']
            gulgast_entries[entry_id] = entry['player_name']
        
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')

    return gulgast_entries

In [9]:
def find_entries_points_in_gameweek(entry_id, gw):
    points = None
    try:
        response = requests.get(url + entry_endpoint + f'{entry_id}/event/{gw}/picks/')
        
        points = int(response.json()['entry_history']['points'])
    
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
            print(f'Other error occurred: {err}')
        
    return points

In [10]:
# finnur stig leikmanns í hlutadeild
def find_entry_points_in_phase(entry_id, phase_id):
    if phase_id > find_current_phase():
        return None
    
    phase_key = str(phase_id)
    
    start = phases[phase_key]['start_event']
    end = phases[str(phase_key)]['end_event']

    if current_gw < end:
        end = current_gw
        
    phase_points = 0
    gw = start
    while gw <= end:
        phase_points += find_entries_points_in_gameweek(entry_id, gw)
        gw += 1
        
    return phase_points

In [11]:
# finnur töfluna fyrir hlutadeild
def find_phase_table(phase_id):
    phase_table = {}
    for entry_id in gulgast_entries:
        points = find_entry_points_in_phase(entry_id, phase_id)
        entry_name = gulgast_entries[entry_id]
        phase_table[entry_name] = points
    
    table_df = pd.DataFrame.from_dict(phase_table, orient='index')
    table_df = table_df.reset_index()
    table_df['Hlutadeild'] = phase_id
    table_df.columns = ['Nafn', 'Stig', 'Hlutadeild']
    table_df = table_df.sort_values('Stig', ascending=False)
    return table_df

In [12]:
# finnur winner hlutadeildar
def find_phase_winner(phase_id):
    if not is_phase_finished(phase_id):
        return None, None
    
    winner_points = -1000
    winner = ''
    for entry_id in gulgast_entries:
        points = find_entry_points_in_phase(entry_id, phase_id)
        name = gulgast_entries[entry_id]
        # Einhverjir jafnir á stigum
        if points == winner_points:
            winner += f', {name}'
        # Einhver hærri    
        if points > winner_points:
            winner = name
            winner_points = points
    
    return winner, winner_points

In [23]:
def get_phases():
    phase_id = 1
    start= 1
    end = 4
    last_gw = 38
    phases = {}

    # finnum upplýsingarnar fyrir hverja hlutadeild
    while end <= last_gw:
        phase = {}
        
        # hlutadeildir 5 og 8 innihalda 5 umferðir
        end = end if (phase_id != 5 and phase_id != 8) else end + 1

        # grunnupplýsingar fyrir hverja hlutadeild
        phase['id'] = phase_id
        phase['name'] = f'hlutadeild {phase_id}'
        phase['start_event'] = start
        phase['end_event'] = end
        phase['is_current'] = True if start <= current_gw <= end else False
        phase['is_finished'] = is_gw_finished(end)

        # bætum hlutadeild í dictið
        phases[f'{phase_id}'] = phase

        # gerum tilbúið fyrir næstu loop-u
        phase_id +=1
        start = end + 1
        end += 4

    return phases

In [14]:
def generate_finished_phases(phases):
    finished_phases = {}
    for key in phases:
        phase = phases[key]
        phase_id = phase['id']
        if phase['is_finished']:
            winner, winner_points = find_phase_winner(phase_id)
            phase['winner'] = winner
            phase['winner_points'] = winner_points
            finished_phases[key] = phase
        else:
            return finished_phases

## Main

In [15]:
current_gw = find_current_gw()
current_gw_finished = is_gw_finished(current_gw)

In [16]:
gulgast_entries = find_gulgast_entries_id()

In [24]:
phases = get_phases()
current_phase = find_current_phase()

In [18]:
current_phase_table = find_phase_table(current_phase)
current_phase_table

,Nafn,Stig,Hlutadeild
0,Daníel Þór Rúnarsson,42,1
1,Viktor Snær Rúnarsson,42,1
2,Breki Einarsson,41,1
3,Jón Kaldal,39,1
4,Thorkell Helgason,38,1
5,Kristþór Logi Sæmundsson,38,1
6,Andrés Burknason,36,1
7,Petur Jokull Thorvaldsson,35,1
8,Kristofer Karl Jensson,35,1
9,Haukur Marian Suska,34,1


In [19]:
current_phase_winner, current_phase_winner_points = find_phase_winner(current_phase)
print(f'Sigurvegari hlutadeildar {current_phase}: {current_phase_winner} með {current_phase_winner_points} stig')

Sigurvegari hlutadeildar 1: None með None stig


In [20]:
finished_phases = generate_finished_phases(phases)
print_json(finished_phases)

{}


### Aðrar upplýsingar

In [21]:
print_json(gulgast_entries)

{
  "6719035": "Daníel Þór Rúnarsson",
  "801840": "Viktor Snær Rúnarsson",
  "6049917": "Breki Einarsson",
  "3857781": "Jón Kaldal",
  "5242555": "Thorkell Helgason",
  "6673584": "Kristþór Logi Sæmundsson",
  "4984408": "Andrés Burknason",
  "5205277": "Petur Jokull Thorvaldsson",
  "7618498": "Kristofer Karl Jensson",
  "5651623": "Haukur Marian Suska"
}


In [25]:
print_json(phases)

{
  "1": {
    "id": 1,
    "name": "hlutadeild 1",
    "start_event": 1,
    "end_event": 4,
    "is_current": true,
    "is_finished": false
  },
  "2": {
    "id": 2,
    "name": "hlutadeild 2",
    "start_event": 5,
    "end_event": 8,
    "is_current": false,
    "is_finished": false
  },
  "3": {
    "id": 3,
    "name": "hlutadeild 3",
    "start_event": 9,
    "end_event": 12,
    "is_current": false,
    "is_finished": false
  },
  "4": {
    "id": 4,
    "name": "hlutadeild 4",
    "start_event": 13,
    "end_event": 16,
    "is_current": false,
    "is_finished": false
  },
  "5": {
    "id": 5,
    "name": "hlutadeild 5",
    "start_event": 17,
    "end_event": 21,
    "is_current": false,
    "is_finished": false
  },
  "6": {
    "id": 6,
    "name": "hlutadeild 6",
    "start_event": 22,
    "end_event": 25,
    "is_current": false,
    "is_finished": false
  },
  "7": {
    "id": 7,
    "name": "hlutadeild 7",
    "start_event": 26,
    "end_event": 29,
    "is_current"